# CNN Classification with CR Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


,sentence,label,split
0,weaknesses are minor the feel and layout of th...,0,train
1,many of our disney movies do n 't play on this...,0,train
2,player has a problem with dual layer dvd 's su...,0,train
3,i know the saying is you get what you pay for ...,0,train
4,will never purchase apex again .,0,train
...,...,...,...
3770,"so far , the anti spam feature seems to be ver...",1,train
3771,i downloaded a trial version of computer assoc...,1,train
3772,i did not have any of the installation problem...,1,train
3773,their products have been great and have saved ...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1368,1368
1,2407,2407


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  will never purchase apex again .
Into a sequence of int: [72, 194, 285, 207, 286]
Into a padded sequence: [ 72 194 285 207 286   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
and 3
i 4
it 5
to 6
a 7
is 8
of 9
this 10
5336


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [247]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [248]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_1012"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1019 (Embedding)   (None, 100, 300)          1530000   
_________________________________________________________________
conv1d_1014 (Conv1D)         (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_1014 (MaxPooli (None, 49, 100)           0         
_________________________________________________________________
flatten_1014 (Flatten)       (None, 4900)              0         
_________________________________________________________________
dropout_2021 (Dropout)       (None, 4900)              0         
_________________________________________________________________
dense_2019 (Dense)           (None, 10)                49010     
_________________________________________________________________
dropout_2022 (Dropout)       (None, 10)            

In [258]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [259]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=15, verbose=2, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/15
68/68 - 5s - loss: 0.6557 - accuracy: 0.6359 - val_loss: 0.6421 - val_accuracy: 0.6270
Epoch 2/15
68/68 - 3s - loss: 0.5799 - accuracy: 0.6479 - val_loss: 0.5172 - val_accuracy: 0.7196
Epoch 3/15
68/68 - 3s - loss: 0.4034 - accuracy: 0.8101 - val_loss: 0.4896 - val_accuracy: 0.7646
Epoch 4/15
68/68 - 3s - loss: 0.2734 - accuracy: 0.8973 - val_loss: 0.4734 - val_accuracy: 0.7804
Epoch 5/15
68/68 - 3s - loss: 0.1999 - accuracy: 0.9320 - val_loss: 0.5114 - val_accuracy: 0.7963
Epoch 6/15
68/68 - 3s - loss: 0.1623 - accuracy: 0.9520 - val_loss: 0.6358 - val_accuracy: 0.7751
Epoch 7/15
68/68 - 3s - loss: 0.1230 - accuracy: 0.9661 - val_loss: 0.7181 - val_accuracy: 0.7831
Epoch 8/15
68/68 - 3s - loss: 0.0856 - accuracy: 0.9756 - val_loss: 0.8507 - val_accuracy: 0.7698
Epoch 9/15
68/68 - 3s - loss: 0.0752 - accuracy: 0.9800 - val_loss: 0.9476 - val_accuracy: 0.7831
Epoch 10/15
68/68 - 3s - loss: 0.0654 - accuracy: 0.9794 - val_loss: 1.0139 - val_accuracy: 0.7751
Restoring model wei

Epoch 9/15
68/68 - 4s - loss: 0.0720 - accuracy: 0.9782 - val_loss: 0.7699 - val_accuracy: 0.8011
Epoch 10/15
68/68 - 4s - loss: 0.0599 - accuracy: 0.9809 - val_loss: 0.8111 - val_accuracy: 0.8011
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 82.22811818122864
Epoch 1/15
68/68 - 4s - loss: 0.6578 - accuracy: 0.6324 - val_loss: 0.6326 - val_accuracy: 0.6711
Epoch 2/15
68/68 - 3s - loss: 0.5823 - accuracy: 0.6342 - val_loss: 0.5137 - val_accuracy: 0.6711
Epoch 3/15
68/68 - 3s - loss: 0.4440 - accuracy: 0.7828 - val_loss: 0.4896 - val_accuracy: 0.7745
Epoch 4/15
68/68 - 3s - loss: 0.3115 - accuracy: 0.8952 - val_loss: 0.5309 - val_accuracy: 0.7745
Epoch 5/15
68/68 - 3s - loss: 0.2441 - accuracy: 0.9341 - val_loss: 0.5694 - val_accuracy: 0.7984
Epoch 6/15
68/68 - 3s - loss: 0.2008 - accuracy: 0.9562 - val_loss: 0.5949 - val_accuracy: 0.7851
Epoch 7/15
68/68 - 3s - loss: 0.1715 - accuracy: 0.9644 - val_loss: 0.6604 - val_accuracy: 0.7851


Epoch 12/15
68/68 - 4s - loss: 0.0828 - accuracy: 0.9965 - val_loss: 0.8773 - val_accuracy: 0.8122
Epoch 13/15
68/68 - 4s - loss: 0.0849 - accuracy: 0.9929 - val_loss: 0.8501 - val_accuracy: 0.7937
Epoch 14/15
68/68 - 4s - loss: 0.0802 - accuracy: 0.9950 - val_loss: 0.9222 - val_accuracy: 0.8069
Epoch 15/15
68/68 - 4s - loss: 0.0728 - accuracy: 0.9965 - val_loss: 0.9144 - val_accuracy: 0.8042
Test Accuracy: 80.42327761650085
Epoch 1/15
68/68 - 5s - loss: 0.6681 - accuracy: 0.6241 - val_loss: 0.6353 - val_accuracy: 0.6640
Epoch 2/15
68/68 - 3s - loss: 0.6240 - accuracy: 0.6347 - val_loss: 0.5481 - val_accuracy: 0.6640
Epoch 3/15
68/68 - 4s - loss: 0.4957 - accuracy: 0.6347 - val_loss: 0.5100 - val_accuracy: 0.6640
Epoch 4/15
68/68 - 4s - loss: 0.3625 - accuracy: 0.8516 - val_loss: 0.5474 - val_accuracy: 0.7831
Epoch 5/15
68/68 - 4s - loss: 0.2441 - accuracy: 0.8893 - val_loss: 0.5706 - val_accuracy: 0.7857
Epoch 6/15
68/68 - 4s - loss: 0.1661 - accuracy: 0.9176 - val_loss: 0.6003 - val_

Epoch 4/15
68/68 - 4s - loss: 0.2767 - accuracy: 0.8955 - val_loss: 0.4368 - val_accuracy: 0.8175
Epoch 5/15
68/68 - 4s - loss: 0.1776 - accuracy: 0.9373 - val_loss: 0.4952 - val_accuracy: 0.8069
Epoch 6/15
68/68 - 4s - loss: 0.1222 - accuracy: 0.9594 - val_loss: 0.6063 - val_accuracy: 0.8042
Epoch 7/15
68/68 - 4s - loss: 0.0852 - accuracy: 0.9661 - val_loss: 0.7405 - val_accuracy: 0.8095
Epoch 8/15
68/68 - 3s - loss: 0.0586 - accuracy: 0.9723 - val_loss: 0.7804 - val_accuracy: 0.8201
Epoch 9/15
68/68 - 3s - loss: 0.0537 - accuracy: 0.9750 - val_loss: 0.9274 - val_accuracy: 0.8201
Epoch 10/15
68/68 - 4s - loss: 0.0493 - accuracy: 0.9738 - val_loss: 0.9827 - val_accuracy: 0.8175
Epoch 11/15
68/68 - 4s - loss: 0.0485 - accuracy: 0.9741 - val_loss: 1.0872 - val_accuracy: 0.8016
Epoch 12/15
68/68 - 4s - loss: 0.0424 - accuracy: 0.9735 - val_loss: 1.1858 - val_accuracy: 0.8122
Epoch 13/15
68/68 - 4s - loss: 0.0327 - accuracy: 0.9823 - val_loss: 1.2833 - val_accuracy: 0.8148
Restoring model 

68/68 - 5s - loss: 0.1265 - accuracy: 0.9709 - val_loss: 0.5807 - val_accuracy: 0.8064
Epoch 7/15
68/68 - 5s - loss: 0.0927 - accuracy: 0.9773 - val_loss: 0.7008 - val_accuracy: 0.7931
Epoch 8/15
68/68 - 5s - loss: 0.0755 - accuracy: 0.9871 - val_loss: 0.7652 - val_accuracy: 0.7851
Epoch 9/15
68/68 - 5s - loss: 0.0795 - accuracy: 0.9865 - val_loss: 0.8609 - val_accuracy: 0.7931
Epoch 10/15
68/68 - 4s - loss: 0.0637 - accuracy: 0.9894 - val_loss: 0.9140 - val_accuracy: 0.7958
Epoch 11/15
68/68 - 5s - loss: 0.0557 - accuracy: 0.9921 - val_loss: 1.0168 - val_accuracy: 0.7851
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 80.63660264015198
Epoch 1/15
68/68 - 7s - loss: 0.6671 - accuracy: 0.6342 - val_loss: 0.6523 - val_accuracy: 0.6446
Epoch 2/15
68/68 - 5s - loss: 0.6155 - accuracy: 0.6368 - val_loss: 0.5317 - val_accuracy: 0.6446
Epoch 3/15
68/68 - 4s - loss: 0.4553 - accuracy: 0.7281 - val_loss: 0.4536 - val_accuracy: 0.8064
Epoch 4/15

...

Epoch 3/15
68/68 - 4s - loss: 0.2578 - accuracy: 0.9111 - val_loss: 0.4495 - val_accuracy: 0.8037
Epoch 4/15
68/68 - 4s - loss: 0.1339 - accuracy: 0.9662 - val_loss: 0.5287 - val_accuracy: 0.7772
Epoch 5/15
68/68 - 4s - loss: 0.0717 - accuracy: 0.9912 - val_loss: 0.5584 - val_accuracy: 0.7851
Epoch 6/15
68/68 - 4s - loss: 0.0419 - accuracy: 0.9959 - val_loss: 0.6373 - val_accuracy: 0.7745
Epoch 7/15
68/68 - 4s - loss: 0.0273 - accuracy: 0.9985 - val_loss: 0.6877 - val_accuracy: 0.7851
Epoch 8/15
68/68 - 4s - loss: 0.0206 - accuracy: 0.9985 - val_loss: 0.7207 - val_accuracy: 0.8117
Epoch 9/15
68/68 - 4s - loss: 0.0194 - accuracy: 0.9991 - val_loss: 0.7451 - val_accuracy: 0.8064
Epoch 10/15
68/68 - 4s - loss: 0.0150 - accuracy: 0.9994 - val_loss: 0.7818 - val_accuracy: 0.8011
Epoch 11/15
68/68 - 4s - loss: 0.0150 - accuracy: 0.9991 - val_loss: 0.8250 - val_accuracy: 0.7825
Epoch 12/15
68/68 - 4s - loss: 0.0129 - accuracy: 0.9994 - val_loss: 0.8518 - val_accuracy: 0.7958
Epoch 13/15
68/68

Epoch 1/15
68/68 - 5s - loss: 0.6582 - accuracy: 0.6265 - val_loss: 0.6124 - val_accuracy: 0.6393
Epoch 2/15
68/68 - 4s - loss: 0.4741 - accuracy: 0.7822 - val_loss: 0.4539 - val_accuracy: 0.7772
Epoch 3/15
68/68 - 4s - loss: 0.2395 - accuracy: 0.9238 - val_loss: 0.5009 - val_accuracy: 0.7533
Epoch 4/15
68/68 - 4s - loss: 0.1338 - accuracy: 0.9635 - val_loss: 0.5769 - val_accuracy: 0.7586
Epoch 5/15
68/68 - 4s - loss: 0.0663 - accuracy: 0.9888 - val_loss: 0.6759 - val_accuracy: 0.7613
Epoch 6/15
68/68 - 4s - loss: 0.0370 - accuracy: 0.9974 - val_loss: 0.7566 - val_accuracy: 0.7692
Epoch 7/15
68/68 - 4s - loss: 0.0260 - accuracy: 0.9985 - val_loss: 0.8213 - val_accuracy: 0.7560
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 77.71883010864258
Epoch 1/15
68/68 - 5s - loss: 0.6699 - accuracy: 0.6204 - val_loss: 0.6112 - val_accuracy: 0.6631
Epoch 2/15
68/68 - 4s - loss: 0.4827 - accuracy: 0.7763 - val_loss: 0.4489 - val_accuracy: 0.7878
E

Test Accuracy: 78.57142686843872
Epoch 1/15
68/68 - 5s - loss: 0.6650 - accuracy: 0.6253 - val_loss: 0.6263 - val_accuracy: 0.6508
Epoch 2/15
68/68 - 4s - loss: 0.4625 - accuracy: 0.7907 - val_loss: 0.4437 - val_accuracy: 0.7804
Epoch 3/15
68/68 - 4s - loss: 0.2420 - accuracy: 0.9217 - val_loss: 0.4595 - val_accuracy: 0.7698
Epoch 4/15
68/68 - 4s - loss: 0.1175 - accuracy: 0.9762 - val_loss: 0.5752 - val_accuracy: 0.7566
Epoch 5/15
68/68 - 4s - loss: 0.0616 - accuracy: 0.9929 - val_loss: 0.6778 - val_accuracy: 0.7619
Epoch 6/15
68/68 - 4s - loss: 0.0390 - accuracy: 0.9965 - val_loss: 0.6796 - val_accuracy: 0.7910
Epoch 7/15
68/68 - 4s - loss: 0.0301 - accuracy: 0.9971 - val_loss: 0.7777 - val_accuracy: 0.7619
Epoch 8/15
68/68 - 4s - loss: 0.0223 - accuracy: 0.9988 - val_loss: 0.8372 - val_accuracy: 0.7672
Epoch 9/15
68/68 - 4s - loss: 0.0209 - accuracy: 0.9974 - val_loss: 0.8812 - val_accuracy: 0.7778
Epoch 10/15
68/68 - 4s - loss: 0.0156 - accuracy: 0.9991 - val_loss: 0.9288 - val_acc

Epoch 2/15
68/68 - 4s - loss: 0.4542 - accuracy: 0.7999 - val_loss: 0.4845 - val_accuracy: 0.7639
Epoch 3/15
68/68 - 4s - loss: 0.2401 - accuracy: 0.9202 - val_loss: 0.5338 - val_accuracy: 0.7851
Epoch 4/15
68/68 - 4s - loss: 0.1132 - accuracy: 0.9762 - val_loss: 0.6017 - val_accuracy: 0.7719
Epoch 5/15
68/68 - 4s - loss: 0.0593 - accuracy: 0.9932 - val_loss: 0.6604 - val_accuracy: 0.7851
Epoch 6/15
68/68 - 4s - loss: 0.0469 - accuracy: 0.9921 - val_loss: 0.7239 - val_accuracy: 0.7825
Epoch 7/15
68/68 - 4s - loss: 0.0301 - accuracy: 0.9971 - val_loss: 0.7374 - val_accuracy: 0.7851
Epoch 8/15
68/68 - 4s - loss: 0.0279 - accuracy: 0.9965 - val_loss: 0.8651 - val_accuracy: 0.7639
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 78.51458787918091

   Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0        relu       1  79.629630  78.306878  77.513230  82.539684  80.158728   
1        relu       2  77.513230  81

68/68 - 4s - loss: 0.0110 - accuracy: 0.9988 - val_loss: 1.0608 - val_accuracy: 0.7751
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 81.4814805984497
Epoch 1/15
68/68 - 5s - loss: 0.6644 - accuracy: 0.6210 - val_loss: 0.6095 - val_accuracy: 0.6578
Epoch 2/15
68/68 - 4s - loss: 0.4625 - accuracy: 0.8058 - val_loss: 0.5079 - val_accuracy: 0.7560
Epoch 3/15
68/68 - 4s - loss: 0.2528 - accuracy: 0.9161 - val_loss: 0.5439 - val_accuracy: 0.7321
Epoch 4/15
68/68 - 4s - loss: 0.1280 - accuracy: 0.9738 - val_loss: 0.6115 - val_accuracy: 0.7507
Epoch 5/15
68/68 - 4s - loss: 0.0735 - accuracy: 0.9862 - val_loss: 0.6559 - val_accuracy: 0.7666
Epoch 6/15
68/68 - 4s - loss: 0.0458 - accuracy: 0.9950 - val_loss: 0.7943 - val_accuracy: 0.7480
Epoch 7/15
68/68 - 4s - loss: 0.0298 - accuracy: 0.9976 - val_loss: 0.8429 - val_accuracy: 0.7613
Epoch 8/15
68/68 - 4s - loss: 0.0219 - accuracy: 0.9994 - val_loss: 0.9081 - val_accuracy: 0.7560
Epoch 9/15
68

## Summary

In [260]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
1,relu,2,77.513230,81.481481,79.629630,80.423278,80.158728,80.106103,79.310346,81.697613,81.962866,81.432360,80.371563
4,relu,5,80.158728,78.306878,81.481481,78.306878,82.539684,83.023870,78.779840,80.371350,81.167108,79.045093,80.318091
3,relu,4,80.158728,79.365081,80.158728,81.216931,80.687833,79.045093,83.023870,80.901855,77.718830,80.901855,80.317881
5,relu,6,80.687833,81.216931,76.190478,80.158728,83.597887,78.779840,81.432360,77.188331,79.840851,83.023870,80.211711
8,tanh,3,83.597887,77.777779,79.365081,80.423278,81.746030,77.984083,79.045093,79.575598,81.167108,79.840851,80.052279
11,tanh,6,80.423278,80.423278,75.661373,80.423278,81.481481,76.657826,84.350133,80.106103,79.045093,81.697613,80.026945
0,relu,1,79.629630,78.306878,77.513230,82.539684,80.158728,78.249335,82.228118,79.840851,80.106103,79.045093,79.761765
9,tanh,4,79.629630,84.391534,78.306878,81.481481,78.835976,77.718830,79.840851,77.718830,81.167108,77.984083,79.707520
6,tanh,1,77.513230,77.777779,78.571427,81.216931,82.010579,78.514588,77.453583,81.697613,81.432360,79.310346,79.549844
10,tanh,5,82.539684,78.571427,79.100531,78.571427,80.158728,79.840851,79.045093,80.106103,78.249335,78.514588,79.469777


In [272]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,80.371563
tanh,80.052279


In [310]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_CR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## CNN Model

In [20]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [21]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          300000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 100)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4900)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4900)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                49010     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)               

## Train and Test the Model

In [22]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [24]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Epoch 1/100
68/68 [==============================] - 3s 26ms/step - loss: 0.6742 - accuracy: 0.6232 - val_loss: 0.6616 - val_accuracy: 0.6032
Epoch 2/100
68/68 [==============================] - 1s 18ms/step - loss: 0.6304 - accuracy: 0.6539 - val_loss: 0.6263 - val_accuracy: 0.6032
Epoch 3/100
68/68 [==============================] - 1s 18ms/step - loss: 0.6030 - accuracy: 0.6651 - val_loss: 0.5539 - val_accuracy: 0.6825
Epoch 4/100
68/68 [==============================] - 1s 20ms/step - loss: 0.5483 - accuracy: 0.7331 - val_loss: 0.4856 - val_accuracy: 0.7619
Epoch 5/100
68/68 [==============================] - 1s 20ms/step - loss: 0.5035 - accuracy: 0.7478 - val_loss: 0.4629 - val_accuracy: 0.7566
Epoch 6/100
68/68 [==============================] - 1s 22ms/step - loss: 0.4345 - accuracy: 0.8021 - val_loss: 0.4586 - val_accuracy: 0.7646
Epoch 7/100
68/68 [==============================] - 1s 18ms/step - loss: 0.3991 - accuracy: 0.8104 - val_loss: 0.4503 - val_accuracy: 0.7751
Epoch 

68/68 [==============================] - 1s 20ms/step - loss: 0.2974 - accuracy: 0.8800 - val_loss: 0.4819 - val_accuracy: 0.7725
Epoch 15/100
68/68 [==============================] - 2s 23ms/step - loss: 0.2883 - accuracy: 0.8799 - val_loss: 0.4890 - val_accuracy: 0.7778
Epoch 16/100
68/68 [==============================] - 2s 22ms/step - loss: 0.2949 - accuracy: 0.8668 - val_loss: 0.4979 - val_accuracy: 0.7778
Epoch 17/100
68/68 [==============================] - 2s 24ms/step - loss: 0.2617 - accuracy: 0.8952 - val_loss: 0.4794 - val_accuracy: 0.7725
Epoch 18/100
68/68 [==============================] - 1s 20ms/step - loss: 0.2648 - accuracy: 0.8987 - val_loss: 0.5037 - val_accuracy: 0.7619
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Test Accuracy: 78.57142686843872
Epoch 1/100
68/68 [==============================] - 3s 27ms/step - loss: 0.6921 - accuracy: 0.6050 - val_loss: 0.6633 - val_accuracy: 0.6366
Epoch 2/100
68/68 [====================

Epoch 11/100
68/68 [==============================] - 1s 19ms/step - loss: 0.3419 - accuracy: 0.8590 - val_loss: 0.4126 - val_accuracy: 0.8037
Epoch 12/100
68/68 [==============================] - 1s 19ms/step - loss: 0.3087 - accuracy: 0.8838 - val_loss: 0.4195 - val_accuracy: 0.8090
Epoch 13/100
68/68 [==============================] - 1s 22ms/step - loss: 0.2890 - accuracy: 0.8975 - val_loss: 0.4244 - val_accuracy: 0.8011
Epoch 14/100
68/68 [==============================] - 2s 22ms/step - loss: 0.2921 - accuracy: 0.8826 - val_loss: 0.4300 - val_accuracy: 0.8090
Epoch 15/100
68/68 [==============================] - 1s 21ms/step - loss: 0.2676 - accuracy: 0.8924 - val_loss: 0.4301 - val_accuracy: 0.7931
Epoch 16/100
68/68 [==============================] - 1s 19ms/step - loss: 0.2532 - accuracy: 0.9031 - val_loss: 0.4320 - val_accuracy: 0.7984
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 82.22811818122864
Epoch 1/100
68/68 [======

Epoch 19/100
68/68 [==============================] - 2s 23ms/step - loss: 0.2512 - accuracy: 0.9005 - val_loss: 0.5123 - val_accuracy: 0.7719
Epoch 20/100
68/68 [==============================] - 1s 19ms/step - loss: 0.2336 - accuracy: 0.9029 - val_loss: 0.4606 - val_accuracy: 0.7905
Epoch 21/100
68/68 [==============================] - 1s 19ms/step - loss: 0.2148 - accuracy: 0.9108 - val_loss: 0.4961 - val_accuracy: 0.7905
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 79.84085083007812
Epoch 1/100
68/68 [==============================] - 2s 24ms/step - loss: 0.6797 - accuracy: 0.6249 - val_loss: 0.6460 - val_accuracy: 0.6313
Epoch 2/100
68/68 [==============================] - 2s 22ms/step - loss: 0.6509 - accuracy: 0.6520 - val_loss: 0.6227 - val_accuracy: 0.6313
Epoch 3/100
68/68 [==============================] - 1s 18ms/step - loss: 0.6198 - accuracy: 0.6373 - val_loss: 0.5838 - val_accuracy: 0.6313
Epoch 4/100
68/68 [=========

Epoch 27/100
68/68 [==============================] - 2s 30ms/step - loss: 0.0955 - accuracy: 0.9715 - val_loss: 0.5213 - val_accuracy: 0.8095
Epoch 28/100
68/68 [==============================] - 2s 30ms/step - loss: 0.0931 - accuracy: 0.9699 - val_loss: 0.5371 - val_accuracy: 0.8069
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Test Accuracy: 82.27513432502747
Epoch 1/100
68/68 [==============================] - 3s 33ms/step - loss: 0.6678 - accuracy: 0.6219 - val_loss: 0.6389 - val_accuracy: 0.6349
Epoch 2/100
68/68 [==============================] - 2s 31ms/step - loss: 0.6144 - accuracy: 0.6566 - val_loss: 0.5478 - val_accuracy: 0.7381
Epoch 3/100
68/68 [==============================] - 2s 28ms/step - loss: 0.5337 - accuracy: 0.7235 - val_loss: 0.4831 - val_accuracy: 0.7619
Epoch 4/100
68/68 [==============================] - 2s 25ms/step - loss: 0.4432 - accuracy: 0.7990 - val_loss: 0.4560 - val_accuracy: 0.7698
Epoch 5/100
68/68 [==========

...

Epoch 25/100
68/68 [==============================] - 6s 88ms/step - loss: 0.0946 - accuracy: 0.9821 - val_loss: 0.8141 - val_accuracy: 0.7666
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Test Accuracy: 81.16710782051086
Epoch 1/100
68/68 [==============================] - 6s 75ms/step - loss: 0.6833 - accuracy: 0.6200 - val_loss: 0.6610 - val_accuracy: 0.6260
Epoch 2/100
68/68 [==============================] - 5s 68ms/step - loss: 0.6355 - accuracy: 0.6338 - val_loss: 0.5827 - val_accuracy: 0.6260
Epoch 3/100
68/68 [==============================] - 5s 68ms/step - loss: 0.5444 - accuracy: 0.6731 - val_loss: 0.5739 - val_accuracy: 0.7003
Epoch 4/100
68/68 [==============================] - 5s 67ms/step - loss: 0.4941 - accuracy: 0.7582 - val_loss: 0.5297 - val_accuracy: 0.7215
Epoch 5/100
68/68 [==============================] - 5s 69ms/step - loss: 0.4053 - accuracy: 0.8219 - val_loss: 0.4823 - val_accuracy: 0.7798
Epoch 6/100
68/68 [===========

68/68 [==============================] - 4s 60ms/step - loss: 0.0889 - accuracy: 0.9767 - val_loss: 0.6086 - val_accuracy: 0.7958
Epoch 16/100
68/68 [==============================] - 4s 60ms/step - loss: 0.0929 - accuracy: 0.9762 - val_loss: 0.6639 - val_accuracy: 0.8090
Epoch 17/100
68/68 [==============================] - 3s 51ms/step - loss: 0.0763 - accuracy: 0.9814 - val_loss: 0.6994 - val_accuracy: 0.8011
Epoch 18/100
68/68 [==============================] - 4s 57ms/step - loss: 0.0726 - accuracy: 0.9767 - val_loss: 0.6909 - val_accuracy: 0.8037
Epoch 19/100
68/68 [==============================] - 4s 55ms/step - loss: 0.0687 - accuracy: 0.9794 - val_loss: 0.6983 - val_accuracy: 0.7958
Epoch 20/100
68/68 [==============================] - 4s 58ms/step - loss: 0.0757 - accuracy: 0.9764 - val_loss: 0.7440 - val_accuracy: 0.8064
Epoch 21/100
68/68 [==============================] - 4s 54ms/step - loss: 0.0670 - accuracy: 0.9803 - val_loss: 0.7862 - val_accuracy: 0.7905
Epoch 22/100

Epoch 23/100
68/68 [==============================] - 4s 53ms/step - loss: 0.0659 - accuracy: 0.9813 - val_loss: 1.0213 - val_accuracy: 0.7460
Epoch 24/100
68/68 [==============================] - 3s 51ms/step - loss: 0.0622 - accuracy: 0.9841 - val_loss: 0.9907 - val_accuracy: 0.7672
Epoch 25/100
68/68 [==============================] - 4s 55ms/step - loss: 0.0570 - accuracy: 0.9799 - val_loss: 1.0549 - val_accuracy: 0.7222
Epoch 26/100
68/68 [==============================] - 3s 47ms/step - loss: 0.0481 - accuracy: 0.9813 - val_loss: 1.1632 - val_accuracy: 0.7646
Epoch 27/100
68/68 [==============================] - 4s 56ms/step - loss: 0.0628 - accuracy: 0.9788 - val_loss: 1.1015 - val_accuracy: 0.7751
Epoch 28/100
68/68 [==============================] - 3s 49ms/step - loss: 0.0489 - accuracy: 0.9866 - val_loss: 1.2086 - val_accuracy: 0.7196
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Test Accuracy: 77.51322984695435
Epoch 1/100
68/68 [======

Epoch 2/100
68/68 [==============================] - 3s 48ms/step - loss: 0.6604 - accuracy: 0.6444 - val_loss: 0.6598 - val_accuracy: 0.6525
Epoch 3/100
68/68 [==============================] - 3s 51ms/step - loss: 0.6479 - accuracy: 0.6312 - val_loss: 0.5883 - val_accuracy: 0.6525
Epoch 4/100
68/68 [==============================] - 3s 50ms/step - loss: 0.5824 - accuracy: 0.6368 - val_loss: 0.5234 - val_accuracy: 0.6525
Epoch 5/100
68/68 [==============================] - 3s 47ms/step - loss: 0.5272 - accuracy: 0.6334 - val_loss: 0.4891 - val_accuracy: 0.6525
Epoch 6/100
68/68 [==============================] - 4s 53ms/step - loss: 0.4701 - accuracy: 0.6742 - val_loss: 0.4867 - val_accuracy: 0.7639
Epoch 7/100
68/68 [==============================] - 3s 45ms/step - loss: 0.4210 - accuracy: 0.8097 - val_loss: 0.4467 - val_accuracy: 0.8064
Epoch 8/100
68/68 [==============================] - 3s 51ms/step - loss: 0.3188 - accuracy: 0.8701 - val_loss: 0.4679 - val_accuracy: 0.7931
Epoch 

Epoch 15/100
68/68 [==============================] - 7s 104ms/step - loss: 0.0862 - accuracy: 0.9745 - val_loss: 0.6153 - val_accuracy: 0.8064
Epoch 16/100
68/68 [==============================] - 7s 104ms/step - loss: 0.0944 - accuracy: 0.9699 - val_loss: 0.6859 - val_accuracy: 0.7905
Epoch 17/100
68/68 [==============================] - 7s 105ms/step - loss: 0.0928 - accuracy: 0.9733 - val_loss: 0.6859 - val_accuracy: 0.7798
Epoch 18/100
68/68 [==============================] - 7s 108ms/step - loss: 0.0724 - accuracy: 0.9793 - val_loss: 0.7225 - val_accuracy: 0.7745
Epoch 19/100
68/68 [==============================] - 7s 105ms/step - loss: 0.0812 - accuracy: 0.9724 - val_loss: 0.7008 - val_accuracy: 0.7692
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Test Accuracy: 81.9628655910492
Epoch 1/100
68/68 [==============================] - 8s 100ms/step - loss: 0.6843 - accuracy: 0.6123 - val_loss: 0.6599 - val_accuracy: 0.6260
Epoch 2/100
68/68 [==

## Summary

In [25]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
2,relu,3,79.894179,82.804233,82.275134,80.423278,79.365081,83.554375,82.493371,78.779840,83.023870,81.432360,81.404572
0,relu,1,82.539684,82.275134,80.687833,81.481481,78.571427,80.901855,82.228118,82.758623,79.840851,82.758623,81.404363
3,relu,4,80.158728,77.777779,80.158728,79.894179,85.714287,81.167108,80.901855,80.636603,81.432360,80.106103,80.794773
6,relu,7,78.835976,81.481481,79.629630,75.661373,76.984125,83.289123,81.167108,79.045093,80.636603,80.901855,79.763237
5,relu,6,79.365081,78.835976,78.042328,77.513230,81.216931,81.697613,81.167108,79.310346,80.371350,79.045093,79.656506
1,relu,2,82.275134,81.746030,81.216931,76.455027,84.391534,77.718830,58.885944,82.228118,82.228118,85.676390,79.282206
7,relu,8,77.513230,76.984125,77.777779,77.513230,81.481481,80.371350,80.901855,79.045093,81.962866,76.923078,79.047409
4,relu,5,79.100531,82.804233,80.158728,76.190478,65.608466,83.023870,80.106103,79.045093,82.493371,80.901855,78.943273


In [26]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,81.404572


In [27]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_CR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [28]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [29]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_85"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_85 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
conv1d_85 (Conv1D)           (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_85 (MaxPooling (None, 49, 100)           0         
_________________________________________________________________
flatten_85 (Flatten)         (None, 4900)              0         
_________________________________________________________________
dropout_170 (Dropout)        (None, 4900)              0         
_________________________________________________________________
dense_170 (Dense)            (None, 10)                49010     
_________________________________________________________________
dropout_171 (Dropout)        (None, 10)              

## Train and Test the Model

In [30]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [31]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Epoch 1/100
68/68 [==============================] - 4s 44ms/step - loss: 0.6780 - accuracy: 0.6240 - val_loss: 0.6595 - val_accuracy: 0.6164
Epoch 2/100
68/68 [==============================] - 3s 37ms/step - loss: 0.6690 - accuracy: 0.6365 - val_loss: 0.6295 - val_accuracy: 0.6164
Epoch 3/100
68/68 [==============================] - 3s 39ms/step - loss: 0.6099 - accuracy: 0.6364 - val_loss: 0.5424 - val_accuracy: 0.7249
Epoch 4/100
68/68 [==============================] - 3s 40ms/step - loss: 0.5386 - accuracy: 0.7243 - val_loss: 0.4680 - val_accuracy: 0.7963
Epoch 5/100
68/68 [==============================] - 3s 38ms/step - loss: 0.3898 - accuracy: 0.8534 - val_loss: 0.4304 - val_accuracy: 0.7963
Epoch 6/100
68/68 [==============================] - 3s 39ms/step - loss: 0.2816 - accuracy: 0.8929 - val_loss: 0.4478 - val_accuracy: 0.8069
Epoch 7/100
68/68 [==============================] - 3s 41ms/step - loss: 0.2103 - accuracy: 0.9329 - val_loss: 0.5102 - val_accuracy: 0.8122
Epoch 

Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 82.0105791091919
Epoch 1/100
68/68 [==============================] - 4s 44ms/step - loss: 0.6613 - accuracy: 0.6117 - val_loss: 0.6446 - val_accuracy: 0.6296
Epoch 2/100
68/68 [==============================] - 3s 38ms/step - loss: 0.6189 - accuracy: 0.6548 - val_loss: 0.5335 - val_accuracy: 0.6958
Epoch 3/100
68/68 [==============================] - 3s 38ms/step - loss: 0.4875 - accuracy: 0.7746 - val_loss: 0.4437 - val_accuracy: 0.7831
Epoch 4/100
68/68 [==============================] - 3s 41ms/step - loss: 0.3582 - accuracy: 0.8500 - val_loss: 0.4162 - val_accuracy: 0.8122
Epoch 5/100
68/68 [==============================] - 3s 39ms/step - loss: 0.2391 - accuracy: 0.9060 - val_loss: 0.4331 - val_accuracy: 0.8360
Epoch 6/100
68/68 [==============================] - 3s 39ms/step - loss: 0.1811 - accuracy: 0.9313 - val_loss: 0.4866 - val_accuracy: 0.8280
Epoch 7/100
68/68 [=============

68/68 [==============================] - 3s 43ms/step - loss: 0.1729 - accuracy: 0.9684 - val_loss: 0.7403 - val_accuracy: 0.7851
Epoch 17/100
68/68 [==============================] - 3s 45ms/step - loss: 0.1616 - accuracy: 0.9677 - val_loss: 0.7948 - val_accuracy: 0.7745
Epoch 18/100
68/68 [==============================] - 3s 43ms/step - loss: 0.1644 - accuracy: 0.9651 - val_loss: 0.8211 - val_accuracy: 0.7745
Epoch 19/100
68/68 [==============================] - 3s 43ms/step - loss: 0.1477 - accuracy: 0.9705 - val_loss: 0.8940 - val_accuracy: 0.7798
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Test Accuracy: 79.84085083007812

  Activation Filters       acc1      acc2       acc3       acc4       acc5  \
0       relu       1  81.481481  81.74603  82.010579  83.597887  80.952382   

       acc6       acc7       acc8      acc9      acc10        AVG  
0  80.37135  83.289123  81.697613  84.08488  79.840851  81.907218  

Epoch 1/100
68/68 [==========

68/68 [==============================] - 3s 50ms/step - loss: 0.0517 - accuracy: 0.9763 - val_loss: 0.9232 - val_accuracy: 0.8254
Epoch 19/100
68/68 [==============================] - 4s 52ms/step - loss: 0.0547 - accuracy: 0.9775 - val_loss: 0.8330 - val_accuracy: 0.8360
Epoch 20/100
68/68 [==============================] - 3s 50ms/step - loss: 0.0516 - accuracy: 0.9801 - val_loss: 0.9580 - val_accuracy: 0.8175
Epoch 21/100
68/68 [==============================] - 3s 51ms/step - loss: 0.0515 - accuracy: 0.9763 - val_loss: 0.8991 - val_accuracy: 0.8148
Epoch 22/100
68/68 [==============================] - 4s 52ms/step - loss: 0.0500 - accuracy: 0.9778 - val_loss: 0.9433 - val_accuracy: 0.8386
Epoch 23/100
68/68 [==============================] - 3s 50ms/step - loss: 0.0467 - accuracy: 0.9780 - val_loss: 1.0265 - val_accuracy: 0.8201
Epoch 24/100
68/68 [==============================] - 3s 51ms/step - loss: 0.0520 - accuracy: 0.9703 - val_loss: 1.1429 - val_accuracy: 0.8175
Restoring mo

68/68 [==============================] - 3s 47ms/step - loss: 0.0769 - accuracy: 0.9711 - val_loss: 0.9494 - val_accuracy: 0.8011
Epoch 22/100
68/68 [==============================] - 3s 48ms/step - loss: 0.0848 - accuracy: 0.9664 - val_loss: 0.9759 - val_accuracy: 0.8011
Epoch 23/100
68/68 [==============================] - 3s 44ms/step - loss: 0.0797 - accuracy: 0.9684 - val_loss: 1.0583 - val_accuracy: 0.7798
Epoch 24/100
68/68 [==============================] - 3s 49ms/step - loss: 0.0733 - accuracy: 0.9717 - val_loss: 1.2620 - val_accuracy: 0.7851
Epoch 25/100
68/68 [==============================] - 3s 47ms/step - loss: 0.0669 - accuracy: 0.9747 - val_loss: 1.1368 - val_accuracy: 0.7931
Epoch 26/100
68/68 [==============================] - 3s 46ms/step - loss: 0.0751 - accuracy: 0.9749 - val_loss: 1.1228 - val_accuracy: 0.8090
Epoch 27/100
68/68 [==============================] - 3s 49ms/step - loss: 0.0618 - accuracy: 0.9765 - val_loss: 1.1709 - val_accuracy: 0.8011
Epoch 28/100

...

Epoch 1/100
68/68 [==============================] - 6s 71ms/step - loss: 0.6831 - accuracy: 0.6196 - val_loss: 0.6283 - val_accuracy: 0.6772
Epoch 2/100
68/68 [==============================] - 5s 66ms/step - loss: 0.6315 - accuracy: 0.6315 - val_loss: 0.4889 - val_accuracy: 0.7063
Epoch 3/100
68/68 [==============================] - 4s 66ms/step - loss: 0.4718 - accuracy: 0.7539 - val_loss: 0.4045 - val_accuracy: 0.8016
Epoch 4/100
68/68 [==============================] - 5s 66ms/step - loss: 0.3090 - accuracy: 0.8748 - val_loss: 0.4013 - val_accuracy: 0.8042
Epoch 5/100
68/68 [==============================] - 5s 67ms/step - loss: 0.1970 - accuracy: 0.9155 - val_loss: 0.5082 - val_accuracy: 0.7884
Epoch 6/100
68/68 [==============================] - 5s 67ms/step - loss: 0.1225 - accuracy: 0.9536 - val_loss: 0.5677 - val_accuracy: 0.7831
Epoch 7/100
68/68 [==============================] - 5s 67ms/step - loss: 0.0914 - accuracy: 0.9678 - val_loss: 0.6854 - val_accuracy: 0.7937
Epoch 

68/68 [==============================] - 6s 91ms/step - loss: 0.0539 - accuracy: 0.9682 - val_loss: 0.6632 - val_accuracy: 0.8568
Epoch 11/100
68/68 [==============================] - 6s 91ms/step - loss: 0.0468 - accuracy: 0.9737 - val_loss: 0.7519 - val_accuracy: 0.8515
Epoch 12/100
68/68 [==============================] - 6s 91ms/step - loss: 0.0365 - accuracy: 0.9795 - val_loss: 0.7813 - val_accuracy: 0.8355
Epoch 13/100
68/68 [==============================] - 6s 93ms/step - loss: 0.0453 - accuracy: 0.9712 - val_loss: 0.9957 - val_accuracy: 0.8170
Epoch 14/100
68/68 [==============================] - 6s 92ms/step - loss: 0.0348 - accuracy: 0.9822 - val_loss: 0.8129 - val_accuracy: 0.8355
Epoch 15/100
68/68 [==============================] - 6s 91ms/step - loss: 0.0339 - accuracy: 0.9829 - val_loss: 0.9683 - val_accuracy: 0.8196
Epoch 16/100
68/68 [==============================] - 6s 92ms/step - loss: 0.0259 - accuracy: 0.9895 - val_loss: 0.8761 - val_accuracy: 0.8515
Epoch 17/100

Epoch 6/100
68/68 [==============================] - 5s 74ms/step - loss: 0.1841 - accuracy: 0.9371 - val_loss: 0.5450 - val_accuracy: 0.7857
Epoch 7/100
68/68 [==============================] - 5s 76ms/step - loss: 0.1246 - accuracy: 0.9649 - val_loss: 0.6023 - val_accuracy: 0.7989
Epoch 8/100
68/68 [==============================] - 5s 74ms/step - loss: 0.0966 - accuracy: 0.9772 - val_loss: 0.6826 - val_accuracy: 0.8069
Epoch 9/100
68/68 [==============================] - 5s 79ms/step - loss: 0.0923 - accuracy: 0.9803 - val_loss: 0.7604 - val_accuracy: 0.8042
Epoch 10/100
68/68 [==============================] - 5s 81ms/step - loss: 0.0731 - accuracy: 0.9862 - val_loss: 0.8436 - val_accuracy: 0.7857
Epoch 11/100
68/68 [==============================] - 5s 79ms/step - loss: 0.0759 - accuracy: 0.9780 - val_loss: 0.8400 - val_accuracy: 0.7937
Epoch 12/100
68/68 [==============================] - 5s 75ms/step - loss: 0.0653 - accuracy: 0.9904 - val_loss: 0.9927 - val_accuracy: 0.7884
Epo

Epoch 13/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0428 - accuracy: 0.9787 - val_loss: 1.2658 - val_accuracy: 0.7692
Epoch 14/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0351 - accuracy: 0.9875 - val_loss: 1.4287 - val_accuracy: 0.7719
Epoch 15/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0285 - accuracy: 0.9865 - val_loss: 1.5296 - val_accuracy: 0.7878
Epoch 16/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0267 - accuracy: 0.9871 - val_loss: 1.5403 - val_accuracy: 0.7639
Epoch 17/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0354 - accuracy: 0.9840 - val_loss: 1.5314 - val_accuracy: 0.7825
Epoch 18/100
68/68 [==============================] - 7s 107ms/step - loss: 0.0287 - accuracy: 0.9849 - val_loss: 1.5289 - val_accuracy: 0.7745
Epoch 19/100
68/68 [==============================] - 7s 106ms/step - loss: 0.0270 - accuracy: 0.9876 - val_loss: 1.5966 - val_accuracy:

Epoch 11/100
68/68 [==============================] - 6s 89ms/step - loss: 0.0627 - accuracy: 0.9909 - val_loss: 0.8156 - val_accuracy: 0.8170
Epoch 12/100
68/68 [==============================] - 6s 91ms/step - loss: 0.0654 - accuracy: 0.9966 - val_loss: 0.8214 - val_accuracy: 0.8117
Epoch 13/100
68/68 [==============================] - 6s 90ms/step - loss: 0.0649 - accuracy: 0.9951 - val_loss: 0.8179 - val_accuracy: 0.8196
Epoch 14/100
68/68 [==============================] - 6s 90ms/step - loss: 0.0570 - accuracy: 0.9941 - val_loss: 0.9337 - val_accuracy: 0.8223
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 83.81962776184082
Epoch 1/100
68/68 [==============================] - 7s 91ms/step - loss: 0.6910 - accuracy: 0.6026 - val_loss: 0.6846 - val_accuracy: 0.6764
Epoch 2/100
68/68 [==============================] - 6s 88ms/step - loss: 0.6851 - accuracy: 0.6292 - val_loss: 0.6748 - val_accuracy: 0.6764
Epoch 3/100
68/68 [========

## Summary

In [32]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
4,relu,5,81.216931,84.391534,82.010579,83.862436,83.068782,79.840851,82.493371,79.310346,82.758623,83.289123,82.224258
0,relu,1,81.481481,81.746030,82.010579,83.597887,80.952382,80.371350,83.289123,81.697613,84.084880,79.840851,81.907218
1,relu,2,82.010579,81.481481,85.185188,79.629630,82.539684,79.575598,81.432360,80.636603,82.493371,82.228118,81.721261
2,relu,3,80.158728,81.746030,83.597887,78.042328,83.862436,80.371350,83.819628,80.901855,82.493371,81.697613,81.669123
3,relu,4,79.100531,80.423278,82.539684,81.746030,82.804233,83.819628,81.697613,82.758623,80.371350,81.167108,81.642808
5,relu,6,85.449737,82.010579,79.894179,81.746030,83.597887,79.575598,81.962866,79.310346,82.228118,80.371350,81.614669
7,relu,8,80.952382,78.835976,80.687833,79.629630,82.010579,78.779840,83.819628,81.962866,83.819628,84.615386,81.511375
6,relu,7,81.216931,80.423278,82.804233,83.597887,77.777779,78.514588,86.472148,80.371350,78.779840,84.615386,81.457342


In [33]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_CR_3.xlsx', sheet_name='dynamic')